In [ ]:
!pip install cirq

  Using cached cirq_core-1.4.1-py3-none-any.whl.metadata (1.8 kB)
Using cached cirq_core-1.4.1-py3-none-any.whl (1.9 MB)
  Attempting uninstall: cirq-core
    Found existing installation: cirq-core 1.3.0.dev20231018023458
    Uninstalling cirq-core-1.3.0.dev20231018023458:
      Successfully uninstalled cirq-core-1.3.0.dev20231018023458
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyliqtr 1.2.0 requires cirq-core==1.3.0.dev20231018023458, but you have cirq-core 1.4.1 which is incompatible.
qualtran 0.2.0 requires cirq-core==1.3.0.dev20231018023458, but you have cirq-core 1.4.1 which is incompatible.


# Cirq

## Example 01

![path3.png](path3.png)



In [ ]:
import cirq
from cirq import H, Z, X, inverse, CX, CCX
s = cirq.Simulator()

def oracle_computation():
    #edge checking
    yield CX(qq[0], qq[3])
    yield CX(qq[1], qq[3])
    yield CX(qq[1], qq[4])
    yield CX(qq[2], qq[4])

    #adding qubit 3
    yield CX(qq[3], qq[5])

    #adding qubit 4
    yield CCX(qq[4], qq[5], qq[6])
    yield CX(qq[4], qq[5])

    #check if b1 is equal to 1 and store the result in the auxillary qubit
    yield CX(qq[6], qq[7])

def oracle():
    yield oracle_computation()
    yield Z(qq[7])
    yield inverse(oracle_computation())

def grover_diffusion():
    yield H.on_each(*(qq[:3]))
    yield X.on_each(*(qq[:3]))
    yield Z(qq[2]).controlled_by(qq[0], qq[1])
    yield X.on_each(*(qq[:3]))
    yield H.on_each(*(qq[:3]))


# the Grover algorithm
qq = cirq.LineQubit.range(8)
circuit = cirq.Circuit()
circuit.append(H.on_each(*(qq[0:3])))

for i in range(1):
    circuit.append(oracle())
    circuit.append(grover_diffusion())

circuit.append(cirq.measure(*qq[0:3], key='result'))

# determine the statistics of the measurements
trials_number = 10_000
samples = s.run(circuit, repetitions=trials_number)

print("Random guess probability:", 1./2**3)

# create an histogram of the result
def bitstring(bits):
    return "".join(str(int(b)) for b in bits)
counts = samples.histogram(key="result",fold_func=bitstring)
for state, c in counts.items():
    print("Probability of obsering", state, ": " ,c/trials_number)

Random guess probability: 0.125
Probability of obsering 101 :  0.5032
Probability of obsering 010 :  0.4968


In [ ]:
print(circuit)

                  ┌──┐                           ┌──┐   ┌──┐   ┌──┐   ┌──┐
0: ───H───@────────────────────────────────────────────────────────────@─────H───X───@───X───H───M('result')───
          │                                                            │             │           │
1: ───H───┼───@────@─────────────────────────────────────@──────@──────┼H────X───────@───X───H───M─────────────
          │   │    │                                     │      │      │             │           │
2: ───H───┼───┼────┼─────@─────────────────────────@─────┼H─────┼X─────┼─────────────@───X───H───M─────────────
          │   │    │     │                         │     │      │      │
3: ───────X───X────┼@────┼────────────────────────@┼─────┼──────X──────X───────────────────────────────────────
                   ││    │                        ││     │
4: ────────────────X┼────X───@───@───@───────@────┼X─────X─────────────────────────────────────────────────────
                    │        │   │ 

In [ ]:
!pip install pyLIQTR

  Using cached cirq_core-1.3.0.dev20231018023458-py3-none-any.whl.metadata (2.1 kB)
Using cached cirq_core-1.3.0.dev20231018023458-py3-none-any.whl (1.8 MB)
  Attempting uninstall: cirq-core
    Found existing installation: cirq-core 1.4.1
    Uninstalling cirq-core-1.4.1:
      Successfully uninstalled cirq-core-1.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cirq 1.4.1 requires cirq-core==1.4.1, but you have cirq-core 1.3.0.dev20231018023458 which is incompatible.
cirq-aqt 1.4.1 requires cirq-core==1.4.1, but you have cirq-core 1.3.0.dev20231018023458 which is incompatible.
cirq-google 1.4.1 requires cirq-core==1.4.1, but you have cirq-core 1.3.0.dev20231018023458 which is incompatible.
cirq-ionq 1.4.1 requires cirq-core==1.4.1, but you have cirq-core 1.3.0.dev20231018023458 which is incompatible.
cirq-pasqal 1.4.1 requires cirq-core==1.4.1, but you hav

In [ ]:
from pyLIQTR.circuits.operators                     import *
from pyLIQTR.BlockEncodings.CarlemanLinearization   import Carleman_Linearization
from pyLIQTR.ProblemInstances.NonlinearODE          import FOperators
from pyLIQTR.utils.circuit_decomposition            import circuit_decompose_multi
from pyLIQTR.utils.global_ancilla_manager           import gam
from pyLIQTR.utils.resource_analysis                import estimate_resources
from pyLIQTR.utils.printing                         import openqasm

from qualtran._infra.gate_with_registers import get_named_qubits
from qualtran.cirq_interop.jupyter_tools import svg_circuit

import matplotlib.pyplot as plt
from qualtran import *
import numpy as np
import cirq
import math

/usr/local/lib/python3.10/dist-packages/cotengra/hyperoptimizers/hyper.py:33: UserWarning: Couldn't import `kahypar` - skipping from default hyper optimizer and using basic `labels` method instead.
  warnings.warn(


In [ ]:
estimate_resources(circuit)

{'LogicalQubits': 8, 'T': 21, 'Clifford': 59}

## Example 02



In [ ]:
import cirq
from cirq import H, Z, X, inverse, CX, CCX
s = cirq.Simulator()

# 0-3: vertices
# 4-7: edge checking
# 8-10: the number
# 11: auxillary

def edge_check(a, b, c):
    yield CX(qq[a], qq[c])
    yield CX(qq[b], qq[c])

def oracle_computation():
    # check all edges
    yield edge_check(0, 2, 4)
    yield edge_check(0, 3, 5)
    yield edge_check(1, 2, 6)
    yield edge_check(1, 3, 7)

    # add outputs of edge checking
    yield CX(qq[4],qq[8])

    yield CCX(qq[5], qq[8], qq[9])
    yield CX(qq[5],qq[8])

    yield CCX(qq[6], qq[8], qq[9])
    yield CX(qq[6],qq[8])

    yield X(qq[10]).controlled_by(qq[7], qq[8],qq[9])
    yield CCX(qq[7], qq[8], qq[9])
    yield CX(qq[7],qq[8])


    # check if number is equal to four
    yield X(qq[8])
    yield X(qq[9])
    yield X(qq[11]).controlled_by(*(qq[8:11]))
    # note we don't have to undo the X gates!

def oracle():
    yield oracle_computation()
    yield Z(qq[11])
    yield inverse(oracle_computation())

def grover_diffusion():
    yield H.on_each(*(qq[:4]))
    yield X.on_each(*(qq[:4]))
    yield Z(qq[3]).controlled_by(*(qq[:3]))
    yield X.on_each(*(qq[:4]))
    yield H.on_each(*(qq[:4]))

# the Grover algorithm
qq = cirq.LineQubit.range(12)
circuit = cirq.Circuit()
circuit.append(H.on_each(*(qq[0:4])))

for i in range(2):
    circuit.append(oracle())
    circuit.append(grover_diffusion())
circuit.append(cirq.measure(*qq[0:4], key='result'))

# determine the statistics of the measurements
trials_number = 10_000
samples = s.run(circuit, repetitions=trials_number)
result = samples.measurements["result"]

print("Random guess probability:", 1/2**4)
# create an histogram of the result
def bitstring(bits):
    return "".join(str(int(b)) for b in bits)
counts = samples.histogram(key="result",fold_func=bitstring)
for state, c in counts.items():
    print("Probability of observing", state, ": " ,c/trials_number)

Random guess probability: 0.0625
Probability of observing 1100 :  0.4724
Probability of observing 0011 :  0.4734
Probability of observing 0000 :  0.0033
Probability of observing 0110 :  0.0036
Probability of observing 0010 :  0.0044
Probability of observing 1101 :  0.005
Probability of observing 0111 :  0.004
Probability of observing 1111 :  0.0046
Probability of observing 0100 :  0.0056
Probability of observing 1010 :  0.0033
Probability of observing 0001 :  0.003
Probability of observing 1110 :  0.0036
Probability of observing 1000 :  0.0032
Probability of observing 1001 :  0.0034
Probability of observing 1011 :  0.0038
Probability of observing 0101 :  0.0034


In [ ]:
estimate_resources(circuit)

{'LogicalQubits': 12, 'T': 14252, 'Clifford': 28596}

## Example 03

In [ ]:
import cirq
from cirq import H, Z, X, inverse, CX, CCX
s = cirq.Simulator()

# 0-4: vertices
# 5-11: edge checking
# 12-14: the number
# 15: auxillary

def edge_check(a, b, c):
    yield CX(qq[a], qq[c])
    yield CX(qq[b], qq[c])

def oracle_computation():
    # check all edges
    yield edge_check(0, 1, 5)
    yield edge_check(0, 3, 6)
    yield edge_check(0, 4, 7)
    yield edge_check(1, 3, 8)
    yield edge_check(1, 4, 9)
    yield edge_check(2, 3, 10)
    yield edge_check(2, 4, 11)

    # add qubit 5
    yield CX(qq[5], qq[12])

    # add qubits 6-7
    for j in range(6,8):
        yield CCX(qq[j], qq[12], qq[13])
        yield CX(qq[j], qq[12])

    # add qubits 8-11
    for j in range(8,12):
        yield X(qq[14]).controlled_by(qq[j], qq[12], qq[13])
        yield CCX(qq[j], qq[12], qq[13])
        yield CX(qq[j], qq[12])

    # set the last bit
    # we do not have to check 12th qubit
    yield CCX(qq[13], qq[14], qq[15])

def oracle():
    yield oracle_computation()
    yield Z(qq[15])
    yield inverse(oracle_computation())

def grover_diffusion():
    yield H.on_each(*(qq[:5]))
    yield X.on_each(*(qq[:5]))
    yield Z(qq[4]).controlled_by(*(qq[:4]))
    yield X.on_each(*(qq[:5]))
    yield H.on_each(*(qq[:5]))


# the Grover algorithm
qq = cirq.LineQubit.range(16)
circuit = cirq.Circuit()
circuit.append(H.on_each(*(qq[0:5])))

for i in range(2):
    circuit.append(oracle())
    circuit.append(grover_diffusion())

circuit.append(cirq.measure(*qq[0:5], key='result'))

# determine the statistics of the measurements
trials_number = 10_000
samples = s.run(circuit, repetitions=trials_number)
result = samples.measurements["result"]

print("Random guess probability:", 1/2**5)
# create an histogram of the result
def bitstring(bits):
    return "".join(str(int(b)) for b in bits)
counts = samples.histogram(key="result",fold_func=bitstring)
for state, c in counts.items():
    print("Probability of observing", state, ": " ,c/trials_number)

Random guess probability: 0.03125
Probability of observing 00011 :  0.4502
Probability of observing 10000 :  0.0041
Probability of observing 10101 :  0.0028
Probability of observing 11100 :  0.4625
Probability of observing 10001 :  0.0024
Probability of observing 10100 :  0.0023
Probability of observing 01110 :  0.0025
Probability of observing 01010 :  0.0026
Probability of observing 11101 :  0.0038
Probability of observing 11010 :  0.0022
Probability of observing 10011 :  0.0025
Probability of observing 00001 :  0.0021
Probability of observing 01111 :  0.0035
Probability of observing 00101 :  0.0029
Probability of observing 11011 :  0.0036
Probability of observing 00010 :  0.0028
Probability of observing 11111 :  0.0035
Probability of observing 01011 :  0.003
Probability of observing 00000 :  0.0032
Probability of observing 11110 :  0.0031
Probability of observing 00110 :  0.0032
Probability of observing 11001 :  0.0025
Probability of observing 10110 :  0.0036
Probability of observing

In [ ]:
estimate_resources(circuit)

{'LogicalQubits': 16, 'T': 30204, 'Clifford': 60513}

# Qiskit

In [ ]:
# Installing qiskit and related libraries

!pip install qiskit
!pip install qiskit-aer
!pip install qiskit[visualization]
!pip install qutip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.4/39.4 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.5/107.5 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pylatexenc: filename=pylatexenc-2.10-py3-none-any.whl size=136817 sha256=e74bbe66a9124601682537dfb65865d87465de93c7e9c8f12912800145d8bdd6
  Stored in directory: /root/.cache/pip/wheels/d3/31/8b/e09b0386afd80cfc556c00408c9aeea5c35c4d484a9c762fd5
Successfully built pylatexenc
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Import necessary modules for qiskit
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_aer import Aer
import qiskit.quantum_info as qi
from qiskit.visualization import plot_histogram
import numpy as np
from qiskit.circuit.library import HGate, MCXGate
from qiskit.quantum_info import random_unitary
from qiskit import QuantumCircuit
from qiskit.visualization import plot_bloch_multivector
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector
from qiskit.visualization import plot_state_qsphere
from qiskit.circuit.library import XGate, ZGate, HGate, CXGate, CCXGate ,mcrz , MCU1Gate , MCXGate
from qutip import *

## Example 01

In [ ]:
from qiskit import QuantumCircuit
from qiskit.circuit.library import HGate, XGate, ZGate, CXGate, CCXGate
from qiskit.visualization import plot_histogram

def oracle_computation(qc):

    qc.cx(0, 3)
    qc.cx(1, 3)
    qc.cx(1, 4)
    qc.cx(2, 4)
    qc.cx(3, 5)
    qc.ccx(4, 5, 6)
    qc.cx(4, 5)
    qc.cx(6, 7)

def inv_oracle_computation(qc):

    qc.cx(6, 7)
    qc.cx(4, 5)
    qc.ccx(4, 5, 6)
    qc.cx(3, 5)
    qc.cx(2, 4)
    qc.cx(1, 4)
    qc.cx(1, 3)
    qc.cx(0, 3)



def oracle(qc):
    oracle_computation(qc)
    qc.z(7)
    inv_oracle_computation(qc)

def grover_diffusion(qc):
    qc.h(range(3))
    qc.x(range(3))
    qc.h(2)
    qc.mcx([0, 1], 2)
    qc.h(2)
    qc.x(range(3))
    qc.h(range(3))


qc = QuantumCircuit(8,3)  # Create a quantum circuit with 8 qubits
qc.h(range(3))          # Apply Hadamard gates to the first 3 qubits

# Apply the oracle and Grover diffusion operator
oracle(qc)
grover_diffusion(qc)

# Measure the first 3 qubits
qc.measure([0, 1, 2], [0, 1, 2])

# Simulate the circuit
backend = Aer.get_backend('qasm_simulator')
trials_number = 100
result = transpile(qc, backend)
job = backend.run(result, shots=trials_number)
result = job.result()
counts = result.get_counts()
print(counts)


{'010': 45, '101': 55}


## Example 02

In [ ]:
from qiskit import QuantumCircuit
from qiskit.circuit.library import XGate, ZGate, HGate, CXGate, CCXGate ,mcrz , MCXGate
from qiskit.quantum_info import Statevector

# 0-3: vertices
# 4-7: edge checking
# 8-10: the number
# 11: auxiliary

def edge_check(qc, a, b, c):
    qc.cx(a, c)
    qc.cx(b, c)


def oracle_computation(qc):
    # Check all edges
    edge_check(qc, 0, 2, 4)
    edge_check(qc, 0, 3, 5)
    edge_check(qc, 1, 2, 6)
    edge_check(qc, 1, 3, 7)

    # Add outputs of edge checking
    qc.cx(4, 8)

    qc.ccx(5, 8, 9)
    qc.cx(5, 8)

    qc.ccx(6, 8, 9)
    qc.cx(6, 8)

    qc.mcx([7, 8, 9], 10)  # Multi-controlled X gate
    qc.ccx(7, 8, 9)
    qc.cx(7, 8)

    # Check if number is equal to four
    qc.x(8)
    qc.x(9)
    qc.mcx([8, 9, 10], 11)
    # Note we don't have to undo the X gates

def inv_edge_check(qc, a, b, c):
    qc.cx(b, c)
    qc.cx(a, c)


def inv_oracle_computation(qc):


    qc.mcx([8, 9, 10], 11)
    # Note we don't have to undo the X gates
    qc.x(9)
    # Check if number is equal to four
    qc.x(8)
    qc.cx(7, 8)
    qc.ccx(7, 8, 9)
    qc.mcx([7, 8, 9], 10)  # Multi-controlled X gate
    qc.cx(6, 8)
    qc.ccx(6, 8, 9)
    qc.cx(5, 8)
    qc.ccx(5, 8, 9)
    # Add outputs of edge checking
    qc.cx(4, 8)
    # Check all edges
    inv_edge_check(qc, 1, 3, 7)
    inv_edge_check(qc, 1, 2, 6)
    inv_edge_check(qc, 0, 3, 5)
    inv_edge_check(qc, 0, 2, 4)



def oracle(qc):
    oracle_computation(qc)
    qc.z(11)
    inv_oracle_computation(qc)


def grover_diffusion(qc):
    qc.h([0, 1, 2, 3])
    qc.x([0, 1, 2, 3])
    qc.h(2)
    mcx = MCXGate(num_ctrl_qubits=2)
    qc.append(mcx, qargs=[0, 1,2])
    qc.h(2)
    qc.x([0, 1, 2, 3])
    qc.h([0, 1, 2, 3])

# Initialize the circuit
qc = QuantumCircuit(12, 4)

# Apply Hadamard gates to the first 4 qubits
qc.h([0, 1, 2, 3])

# Grover iterations
for i in range(2):
    oracle(qc)
    grover_diffusion(qc)

# Measure the first 4 qubits
qc.measure([0, 1, 2, 3], [0, 1, 2, 3])

# Simulate the circuit
backend = Aer.get_backend('qasm_simulator')
trials_number = 100
# result = execute(qc, backend, shots=trials_number).result()
result = transpile(qc, backend)
job = backend.run(result, shots=trials_number)
result = job.result()
counts = result.get_counts()
# print(counts)
for state, c in counts.items():
    print("Probability of observing", state, ": " ,c/trials_number)

## Example 03

In [ ]:
from qiskit import QuantumCircuit
from qiskit.circuit.library import XGate, ZGate, HGate, CXGate, CCXGate ,mcrz , MCXGate
from qiskit.quantum_info import Statevector

def edge_check(qc, a, b, c):
    qc.cx(a, c)
    qc.cx(b, c)

def inv_edge_check(qc, a, b, c):
    qc.cx(b, c)
    qc.cx(a, c)


def oracle_computation():
    # check all edges
    edge_check(0, 1, 5)
    edge_check(0, 3, 6)
    edge_check(0, 4, 7)
    edge_check(1, 3, 8)
    edge_check(1, 4, 9)
    edge_check(2, 3, 10)
    edge_check(2, 4, 11)

    qc.cx(5, 12)

    for j in range(6,8):
        qc.ccx(j, [12, 13])
        qc.cx(j, 12)

    for j in range(8,12):
      mcx = MCXGate(num_ctrl_qubits=2)
      qc.append(mcx, qargs=[j,12, 13,14])
      qc.ccx(j, [12,13])
      qc.cx(j, 12)

      qc.ccx(13, [14,15])





def oracle():
    oracle_computation(qc)
    qc.z(15)
    inv_oracle_computation(qc)


def grover_diffusion(qc):
    qc.h([0, 1, 2, 3,4])
    qc.x([0, 1, 2, 3,4])
    qc.h(4)
    mcx = MCXGate(num_ctrl_qubits=4)
    qc.append(mcx, qargs=[0, 1,2,3,4])
    qc.h(4)
    qc.x([0, 1, 2, 3,4])
    qc.h([0, 1, 2, 3,4])

In [ ]:
# Measure the first 4 qubits
qc.measure([0, 1, 2, 3,4], [0, 1, 2, 3,4])

# Simulate the circuit
backend = Aer.get_backend('qasm_simulator')
trials_number = 100
# result = execute(qc, backend, shots=trials_number).result()
result = transpile(qc, backend)
job = backend.run(result, shots=trials_number)
result = job.result()
counts = result.get_counts()
# print(counts)
for state, c in counts.items():
    print("Probability of observing", state, ": " ,c/trials_number)

## Extras

In [ ]:
from qiskit import QuantumCircuit
from qiskit.circuit.library import XGate, ZGate, HGate, CXGate, CCXGate ,mcrz , MCU1Gate , MCXGate
from qiskit.quantum_info import Statevector

def inv_edge_check(qc, a, b, c):
    qc.cx(b, c)
    qc.cx(a, c)


def inv_oracle_computation(qc):


    qc.mcx([8, 9, 10], 11)
    # Note we don't have to undo the X gates
    qc.x(9)
    # Check if number is equal to four
    qc.x(8)
    qc.cx(7, 8)
    qc.ccx(7, 8, 9)
    qc.mcx([7, 8, 9], 10)  # Multi-controlled X gate
    qc.cx(6, 8)
    qc.ccx(6, 8, 9)
    qc.cx(5, 8)
    qc.ccx(5, 8, 9)
    # Add outputs of edge checking
    qc.cx(4, 8)
    # Check all edges
    inv_edge_check(qc, 1, 3, 7)
    inv_edge_check(qc, 1, 2, 6)
    inv_edge_check(qc, 0, 3, 5)
    inv_edge_check(qc, 0, 2, 4)



# 0-3: vertices
# 4-7: edge checking
# 8-10: the number
# 11: auxiliary

# def edge_check(qc, a, b, c):
#     yield qc.cx(a, c)
#     yield qc.cx(b, c)

def edge_check(qc, a, b, c):
    qc.cx(a, c)
    qc.cx(b, c)

# def oracle_computation(qc):
#     # Check all edges
#     yield edge_check(qc, 0, 2, 4)
#     yield edge_check(qc, 0, 3, 5)
#     yield edge_check(qc, 1, 2, 6)
#     yield edge_check(qc, 1, 3, 7)


    # Add outputs of edge checking
    # yield qc.cx(4, 8)

    # yield qc.ccx(5, 8, 9)
    # yield qc.cx(5, 8)

    # yield qc.ccx(6, 8, 9)
    # yield qc.cx(6, 8)

    # yield qc.mcx([7, 8, 9], 10)  # Multi-controlled X gate
    # yield qc.ccx(7, 8, 9)
    # yield qc.cx(7, 8)

    # # Check if number is equal to four
    # yield qc.x(8)
    # yield qc.x(9)
    # yield qc.mcx([8, 9, 10], 11)
    # Note we don't have to undo the X gates

def oracle_computation(qc):
    # Check all edges
    edge_check(qc, 0, 2, 4)
    edge_check(qc, 0, 3, 5)
    edge_check(qc, 1, 2, 6)
    edge_check(qc, 1, 3, 7)

    # Add outputs of edge checking
    qc.cx(4, 8)

    qc.ccx(5, 8, 9)
    qc.cx(5, 8)

    qc.ccx(6, 8, 9)
    qc.cx(6, 8)

    qc.mcx([7, 8, 9], 10)  # Multi-controlled X gate
    qc.ccx(7, 8, 9)
    qc.cx(7, 8)

    # Check if number is equal to four
    qc.x(8)
    qc.x(9)
    qc.mcx([8, 9, 10], 11)
    # Note we don't have to undo the X gates

# def oracle(qc):
#     yield oracle_computation(qc)
#     yield qc.z(11)
#     yield inverse(oracle_computation(qc))

def oracle(qc):
  oracle_computation(qc)
  qc.z(11)
  inv_oracle_computation(qc)

def grover_diffusion(qc):
    qc.h([0, 1, 2, 3])
    qc.x([0, 1, 2, 3])
    qc.h(2)
    mcx = MCXGate(num_ctrl_qubits=2)
    qc.append(mcx, qargs=[0, 1,2])
    qc.h(2)
    # qc.mcrz(0, [0, 1, 2], 3)  # Multi-controlled RZ gate
    qc.x([0, 1, 2, 3])
    qc.h([0, 1, 2, 3])


# Initialize the circuit
qc = QuantumCircuit(12, 3)

# Apply Hadamard gates to the first 4 qubits
qc.h([0, 1, 2, 3])

# Grover iterations
for i in range(1):
    oracle(qc)
    grover_diffusion(qc)

# Measure the first 4 qubits
qc.measure([0, 1, 2], [0, 1, 2])

# Simulate the circuit
backend = Aer.get_backend('qasm_simulator')
trials_number = 100
# result = execute(qc, backend, shots=trials_number).result()
result = transpile(qc, backend)
job = backend.run(result, shots=trials_number)
result = job.result()
counts = result.get_counts()
print(counts)
# print("Random guess probability:", 1/2**4)
# # Print the histogram results
# for state, c in counts.items():
#     print(f"Probability of observing {state} : {c/trials_number}")


In [ ]:
def edge_check(qc,a, b, c):
  qc.cx(a, c)
  qc.cx(b, c)
  yield qc


# Initialize the circuit
qc = QuantumCircuit(4, 3)
edge_check(qc,0, 1, 2)
print(qc)

               
q_0: ──■───────
       │       
q_1: ──┼────■──
     ┌─┴─┐┌─┴─┐
q_2: ┤ X ├┤ X ├
     └───┘└───┘
q_3: ──────────
               
c: 3/══════════
               


In [ ]:
from qiskit.circuit.library import XGate, ZGate, HGate, CXGate, CCXGate ,mcrz , MCU1Gate , MCXGate

# Initialize the circuit
qc = QuantumCircuit(4, 3)
# Create a multi-controlled X gate with 3 control qubits
mcx = MCXGate(num_ctrl_qubits=3)
# Apply the MCX gate with qubits 0, 1, and 2 as controls and qubit 3 as the target
qc.h(3)
qc.append(mcx, qargs=[0, 1, 2, 3])
qc.h(3)
# oracle_computation(qc)
# inv_oracle_computation(qc)
print(qc)


qc.measure([0, 1, 2], [0, 1, 2])
# Simulate the circuit
backend = Aer.get_backend('qasm_simulator')
trials_number = 100
# result = execute(qc, backend, shots=trials_number).result()
result = transpile(qc, backend)
job = backend.run(result, shots=trials_number)
result = job.result()
counts = result.get_counts()
print(counts)

In [ ]:
def pra(qc,a, b, c):
  qc.cx(a, c)
  yield qc

In [ ]:
from qiskit import QuantumCircuit

def edge_check(a,b):
    qc.cx(a,b)
    yield qc

num_qubits = 3
qc = QuantumCircuit(num_qubits)
for i in edge_check(0,1):
  None
for i in edge_check(0,1):
  None
print(qc)

               
q_0: ──■────■──
     ┌─┴─┐┌─┴─┐
q_1: ┤ X ├┤ X ├
     └───┘└───┘
q_2: ──────────
               


# QSharp

In [ ]:
# Installing qsharp
!pip install qsharp
!pip install qsharp_widgets

In [ ]:
# importing necessary modules for qsharp
import qsharp
from qsharp.utils import dump_operation
from qsharp_widgets import Circuit
from qsharp_widgets import SpaceChart, EstimateDetails

In [ ]:
%%qsharp

open Microsoft.Quantum.Intrinsic;
open Microsoft.Quantum.Canon;
open Microsoft.Quantum.Math as Math;
open Microsoft.Quantum.Math;
open Microsoft.Quantum.Unstable.StatePreparation;
open Microsoft.Quantum.Convert;


operation oracle_computation(q: Qubit): Unit is Adj + Ctl{

    #edge checking
    CX(qq[0], qq[3])
    CX(qq[1], qq[3])
    CX(qq[1], qq[4])
    CX(qq[2], qq[4])

    #adding qubit 3
    CX(qq[3], qq[5])

    #adding qubit 4
    CCX(qq[4], qq[5], qq[6])
    CX(qq[4], qq[5])

    #check if b1 is equal to 1 and store the result in the auxillary qubit
    CX(qq[6], qq[7])
 }

operation oracle(qq: Qubit[]): Unit is Adj + Ctl{

    oracle_computation()
    Z(qq[7])
    inverse(oracle_computation())
}

operation grover_diffusion(qq: Qubit[]): Unit is Adj + Ctl{

    H(qq[0], qq[1], qq[2])
    X(qq[0], qq[1], qq[2])
    //Z(qq[2]).controlled_by(qq[0], qq[1])
    H(qq[0], qq[1], qq[2])
    X(qq[0], qq[1], qq[2])
}


operation MaxCut() : Result {

for i in range(1):
    oracle()
    grover_diffusion()

}



circuit.append(cirq.measure(*qq[0:3], key='result'))

# determine the statistics of the measurements
trials_number = 10_000
samples = s.run(circuit, repetitions=trials_number)

print("Random guess probability:", 1./2**3)

# create an histogram of the result
def bitstring(bits):
    return "".join(str(int(b)) for b in bits)
counts = samples.histogram(key="result",fold_func=bitstring)
for state, c in counts.items():
    print("Probability of obsering", state, ": " ,c/trials_number)

operation PrepareMyStateQSharp(amplitudes:Double[]) :Unit {

        let N = Length(amplitudes);
        let n = Log(IntAsDouble(N))/Log(2.0);
        use qubits = Qubit[Round(n)];
        PreparePureStateD(amplitudes, qubits);
        let p = Microsoft.Quantum.Diagnostics.DumpMachine();
        ResetAll(qubits);
         MeasureEachZ(qubits);
        return p
    }